In [ ]:
from bs4 import BeautifulSoup # For HTML parsing
from time import sleep # To prevent overwhelming the server between connections
import pandas as pd # For converting results to a dataframe and bar chart plots
# For Visualizations
import numpy as np
import matplotlib.pyplot as plt
import plotly.offline as py
import plotly.graph_objs as go
%matplotlib inline

In [14]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from webdriver_manager.chrome import ChromeDriverManager
driver = webdriver.Chrome(ChromeDriverManager().install())
driver.get('https://condos.ca')
## You will need to create an account and login 


[WDM] - 

[WDM] - ====== WebDriver manager ======
[WDM] - Current google-chrome version is 97.0.4692
[WDM] - Get LATEST driver version for 97.0.4692
[WDM] - Driver [C:\Users\perry\.wdm\drivers\chromedriver\win32\97.0.4692.71\chromedriver.exe] found in cache


In [10]:
#For simplicity, this scrapes downtown toronto condos that were sold in 2020 only.
dt_sold_prices=[]
dt_sold_location=[]
dt_sold_sqft=[]
dt_sold_bd=[]
dt_sold_all=[]
dt_sold_date=[]

def get_page(year,page):
    #The URL below already filters for downtown toronto condos only that are at least 300 sq ft,that were sold in 2020
    url= f'https://condos.ca/toronto/condos-for-sale?size_range=300%2C999999999&property_type=Condo%20Apt%2CComm%20Element%20Condo%2CLeasehold%20Condo&mode=Sold&end_date_unix=exact%2C{year}&sublocality_id=14&page={page}'
    driver.get(url) 
    sleep(3)
    page_source = driver.page_source
    soup = BeautifulSoup(page_source, 'lxml')
    return soup

#attempt to scrape 12 pages.
for year in range (2010,2022):
    for page in range(12):
        soup = get_page(year,page)

        #Scrape prices
        for tag in soup.find_all('div',class_ = 'styles___AskingPrice-sc-54qk44-4 styles___ClosePrice-sc-54qk44-5 dHPUdq hwkkXU'):
            dt_sold_prices.append(tag.get_text())

        #scrape location
        for tag in soup.find_all('address',class_ = 'styles___Address-sc-54qk44-13 gTwVlm'):
            dt_sold_location.append(tag.get_text())

        #scrape sq ft
        for tag in soup.find_all(class_ = 'styles___Size-sc-54qk44-8 KRKbD'):
            dt_sold_sqft.append(tag.get_text())

        #scrape bedrooms    
        for tag in soup.find_all(class_ = 'styles___InfoHolder-sc-54qk44-7 jtFhfz'):
            bedrooms = tag.get_text()[0:3]
            dt_sold_bd.append(bedrooms)

        #scrape all other containers that includes parking
        for tag in soup.find_all(class_ = 'styles___InfoHolder-sc-54qk44-7 jtFhfz'):
            everything = tag.get_text()[0:]
            dt_sold_all.append(everything)

        #scrape sold date
        for tag in soup.find_all(class_ = 'styles___Banner-sc-1a2baz3-0 iWaXsf'):
            everything = tag.get_text()[0:]
            dt_sold_date.append(everything)

In [11]:
#combine all scrape lists into a dictionary so that it cane be transformed to a dataframe
a = {'Prices':dt_sold_prices,
                         'Location':dt_sold_location,
                         'Square Ft':dt_sold_sqft,
                         'Bedrooms':dt_sold_bd,
                         'Everything':dt_sold_all,
                         'Sold Date':dt_sold_date}

df_sold_condos = pd.DataFrame.from_dict(a, orient='index')
df_sold_condos = df_sold_condos.transpose()

In [12]:
#add a column that tags the Downtown region
df_sold_condos['Region']= 'Downtown'
df_sold_condos

# !!! There should be 48 listings per page * 11 pages = ~528 records, but it's not picking up everything. 
#Sometimes it scrapes even less than 213 rows. Not consistent when trying to apply to other regions/years.

,Prices,Location,Square Ft,Bedrooms,Everything,Sold Date,Region
0,"$845,000 $899,000","80 Front St E, Unit 718","1,500-2,000 sqft",2+1,"2+1BD2BA1 Parking1,500-2,000 sqft",Sold in 2010,Downtown
1,"$458,000 $478,000","628 Fleet St, Unit 414","900-1,099 sqft",2BD,"2BD2BA1 Parking900-1,099 sqft",Sold in 2010,Downtown
2,"$450,000 $460,000","21 Carlton St, Unit 1405",700-899 sqft,2BD,2BD2BA1 Parking700-899 sqft,Sold in 2010,Downtown
3,"$355,000 $369,900","110 Charles St E, Unit 2305",500-699 sqft,1BD,1BD1BA0 Parking500-699 sqft,Sold in 2010,Downtown
4,"$347,500 $355,000","18 Beverley St, Unit 109","900-1,099 sqft",2BD,"2BD2BA1 Parking900-1,099 sqft",Sold in 2010,Downtown
...,...,...,...,...,...,...,...
6583,"$590,000 $575,000",351 - 313 Richmond St E,None,1+1,1+1BD1BA1 Parking600-699 sqft,Sold in 2021,Downtown
6584,"$590,000 $599,900",323 - 270 Wellington St W,None,1+1,1+1BD1BA0 Parking600-699 sqft,Sold in 2021,Downtown
6585,"$580,000 $599,900",LPH16 - 320 Richmond St E,None,1+1,1+1BD1BA1 Parking500-599 sqft,Sold in 2021,Downtown
6586,"$451,000 $449,900",710 - 251 Jarvis St,None,Stu,Studio1BA0 Parking0-499 sqft,Sold in 2021,Downtown


In [15]:
sc_sold_prices=[]
sc_sold_location=[]
sc_sold_sqft=[]
sc_sold_bd=[]
sc_sold_all=[]
sc_sold_date=[]

def get_page(year,page):
    url= f'https://condos.ca/toronto/condos-for-sale?size_range=300%2C999999999&property_type=Condo%20Apt%2CComm%20Element%20Condo%2CLeasehold%20Condo&mode=Sold&end_date_unix=exact%2C{year}&sublocality_id=15&page={page}'
    driver.get(url) 
    sleep(3)
    page_source = driver.page_source
    soup = BeautifulSoup(page_source, 'lxml')
    return soup

# for year in range(2011,2012):
for year in range (2010,2022):
    for page in range(12):
        soup = get_page(year,page)
        for tag in soup.find_all('div',class_ = 'styles___AskingPrice-sc-54qk44-4 styles___ClosePrice-sc-54qk44-5 dHPUdq hwkkXU'):
            sc_sold_prices.append(tag.get_text())

        for tag in soup.find_all('address',class_ = 'styles___Address-sc-54qk44-13 gTwVlm'):
            sc_sold_location.append(tag.get_text())

        for tag in soup.find_all(class_ = 'styles___Size-sc-54qk44-8 KRKbD'):
            sc_sold_sqft.append(tag.get_text())


        for tag in soup.find_all(class_ = 'styles___InfoHolder-sc-54qk44-7 jtFhfz'):
            bedrooms = tag.get_text()[0:3]
            sc_sold_bd.append(bedrooms)

        for tag in soup.find_all(class_ = 'styles___InfoHolder-sc-54qk44-7 jtFhfz'):
            everything = tag.get_text()[0:]
            sc_sold_all.append(everything)

        for tag in soup.find_all(class_ = 'styles___Banner-sc-1a2baz3-0 iWaXsf'):
            everything = tag.get_text()[0:]
            sc_sold_date.append(everything)

In [16]:
b = {'Prices':sc_sold_prices,
                         'Location':sc_sold_location,
                         'Square Ft':sc_sold_sqft,
                         'Bedrooms':sc_sold_bd,
                         'Everything':sc_sold_all,
                         'Sold Date':sc_sold_date}

sc_sold_condos = pd.DataFrame.from_dict(b, orient='index')
sc_sold_condos = sc_sold_condos.transpose()

sc_sold_condos['Region']= 'Scarborough'
sc_sold_condos #DONE

,Prices,Location,Square Ft,Bedrooms,Everything,Sold Date,Region
0,"$227,000 $233,000","275 Bamburgh Circ, Unit 1212",700-899 sqft,1+1,1+1BD1BA1 Parking700-899 sqft,Sold in 2010,Scarborough
1,"$212,000 $218,000","350 Alton Towers Circ, Unit 305",,2+1,2+1BD2BA1 Parking,Sold in 2010,Scarborough
2,"$172,000 $176,900","10 Dean Park Rd, Unit 701","1,100-1,299 sqft",2BD,"2BD2BA1 Parking1,100-1,299 sqft",Sold in 2010,Scarborough
3,"$149,000 $155,000","430 Mclevin Ave, Unit 314",,2+1,2+1BD1BA1 Parking,Sold in 2010,Scarborough
4,"$145,000 $149,000","410 Mclevin Ave, Unit 1806",,2BD,2BD2BA1 Parking,Sold in 2010,Scarborough
...,...,...,...,...,...,...,...
6583,"$470,000 $449,900",1002 - 3420 Eglinton Ave,"1,200-1,399 sqft",3BD,"3BD2BA1 Parking1,200-1,399 sqft",Sold in 2021,Scarborough
6584,"$530,000 $539,000",733 - 18 Mondeo Dr,600-699 sqft,1+1,1+1BD1BA1 Parking600-699 sqft,Sold in 2021,Scarborough
6585,"$565,000 $529,900",620 - 2550 Lawrence Ave E,900-999 sqft,2BD,2BD2BA1 Parking900-999 sqft,Sold in 2021,Scarborough
6586,"$575,000 $579,800",3015 - 275 Village Green Sq,600-699 sqft,1+1,1+1BD2BA1 Parking600-699 sqft,Sold in 2021,Scarborough


In [17]:
et_sold_prices=[]
et_sold_location=[]
et_sold_sqft=[]
et_sold_bd=[]
et_sold_all=[]
et_sold_date=[]

def get_page(year,page):
    url= f'https://condos.ca/toronto/condos-for-sale?size_range=300%2C999999999&property_type=Condo%20Apt%2CComm%20Element%20Condo%2CLeasehold%20Condo&mode=Sold&end_date_unix=exact%2C{year}&sublocality_id=16&page={page}'
    driver.get(url) 
    sleep(3)
    page_source = driver.page_source
    soup = BeautifulSoup(page_source, 'lxml')
    return soup

for year in range (2010,2022):
    for page in range(12):
        soup = get_page(year,page)
        for tag in soup.find_all('div',class_ = 'styles___AskingPrice-sc-54qk44-4 styles___ClosePrice-sc-54qk44-5 dHPUdq hwkkXU'):
            et_sold_prices.append(tag.get_text())

        for tag in soup.find_all('address',class_ = 'styles___Address-sc-54qk44-13 gTwVlm'):
            et_sold_location.append(tag.get_text())


        for tag in soup.find_all(class_ = 'styles___Size-sc-54qk44-8 KRKbD'):
            et_sold_sqft.append(tag.get_text())


        for tag in soup.find_all(class_ = 'styles___InfoHolder-sc-54qk44-7 jtFhfz'):
            bedrooms = tag.get_text()[0:3]
            et_sold_bd.append(bedrooms)

        for tag in soup.find_all(class_ = 'styles___InfoHolder-sc-54qk44-7 jtFhfz'):
            everything = tag.get_text()[0:]
            et_sold_all.append(everything)

        for tag in soup.find_all(class_ = 'styles___Banner-sc-1a2baz3-0 iWaXsf'):
            everything = tag.get_text()[0:]
            et_sold_date.append(everything)

In [18]:
c = {'Prices':et_sold_prices,
                         'Location':et_sold_location,
                         'Square Ft':et_sold_sqft,
                         'Bedrooms':et_sold_bd,
                         'Everything':et_sold_all,
                         'Sold Date':et_sold_date}

et_sold_condos = pd.DataFrame.from_dict(c, orient='index')
et_sold_condos = et_sold_condos.transpose()

et_sold_condos['Region']= 'Etobicoke'
et_sold_condos

,Prices,Location,Square Ft,Bedrooms,Everything,Sold Date,Region
0,"$320,000 $329,000","1040 The Queensway Ave, Unit 108",,2BD,2BD2BA1 Parking,Sold in 2010,Etobicoke
1,"$282,500 $284,900","245 Dalesford Rd, Unit 118",,1+1,1+1BD1BA1 Parking,Sold in 2010,Etobicoke
2,"$202,000 $210,000","710 Humberwood Blvd, Unit 2508",,1+1,1+1BD1BA1 Parking,Sold in 2010,Etobicoke
3,"$150,000 $169,500","21 Markbrook Lane, Unit 703",,2+1,2+1BD2BA1 Parking,Sold in 2010,Etobicoke
4,"$302,000 $299,000","225 Sherway Gardens Rd S, Unit 509",700-899 sqft,2BD,2BD2BA1 Parking700-899 sqft,Sold in 2010,Etobicoke
...,...,...,...,...,...,...,...
6583,"$479,900 $479,900",1431 - 700 Humberwood Blvd,700-799 sqft,1BD,1BD1BA1 Parking500-599 sqft,Sold in 2021,Etobicoke
6584,"$585,000 $549,900",1409 - 3865 Lake Shore Blvd W,600-699 sqft,1BD,1BD2BA1 Parking700-799 sqft,Sold in 2021,Etobicoke
6585,"$665,000 $699,000",1403 - 65 Annie Craig Dr W,"1,000-1,199 sqft",1+1,1+1BD1BA1 Parking600-699 sqft,Sold in 2021,Etobicoke
6586,"$1,035,000 $1,099,900",PH06 - 1135 Royal York Rd,600-699 sqft,2+1,"2+1BD3BA1 Parking1,000-1,199 sqft",Sold in 2021,Etobicoke


In [19]:
ny_sold_prices=[]
ny_sold_location=[]
ny_sold_sqft=[]
ny_sold_bd=[]
ny_sold_all=[]
ny_sold_date=[]

def get_page(year,page):
    url= f'https://condos.ca/toronto/condos-for-sale?size_range=300%2C999999999&property_type=Condo%20Apt%2CComm%20Element%20Condo%2CLeasehold%20Condo&mode=Sold&end_date_unix=exact%2C{year}&sublocality_id=17&page={page}'
    driver.get(url)
    sleep(3)
    page_source = driver.page_source
    soup = BeautifulSoup(page_source, 'lxml')
    return soup

for year in range(2010,2022):
    for page in range(12):
        soup = get_page(year,page)
        for tag in soup.find_all('div',class_ = 'styles___AskingPrice-sc-54qk44-4 styles___ClosePrice-sc-54qk44-5 dHPUdq hwkkXU'):
            ny_sold_prices.append(tag.get_text())

        for tag in soup.find_all('address',class_ = 'styles___Address-sc-54qk44-13 gTwVlm'):
            ny_sold_location.append(tag.get_text())


        for tag in soup.find_all(class_ = 'styles___Size-sc-54qk44-8 KRKbD'):
            ny_sold_sqft.append(tag.get_text())


        for tag in soup.find_all(class_ = 'styles___InfoHolder-sc-54qk44-7 jtFhfz'):
            bedrooms = tag.get_text()[0:3]
            ny_sold_bd.append(bedrooms)

        for tag in soup.find_all(class_ = 'styles___InfoHolder-sc-54qk44-7 jtFhfz'):
            everything = tag.get_text()[0:]
            ny_sold_all.append(everything)

        for tag in soup.find_all(class_ = 'styles___Banner-sc-1a2baz3-0 iWaXsf'):
            everything = tag.get_text()[0:]
            ny_sold_date.append(everything)

In [20]:
d = {'Prices':ny_sold_prices,
                         'Location':ny_sold_location,
                         'Square Ft':ny_sold_sqft,
                         'Bedrooms':ny_sold_bd,
                         'Everything':ny_sold_all,
                         'Sold Date':ny_sold_date}

ny_sold_condos = pd.DataFrame.from_dict(d, orient='index')
ny_sold_condos = ny_sold_condos.transpose()

ny_sold_condos['Region']= 'North York'
ny_sold_condos

,Prices,Location,Square Ft,Bedrooms,Everything,Sold Date,Region
0,"$466,000 $469,900","15 Greenview Ave, Unit 1413","900-1,099 sqft",2+1,"2+1BD2BA1 Parking900-1,099 sqft",Sold in 2010,North York
1,"$305,000 $319,000","23 Sheppard Ave E, Unit 608",500-699 sqft,1BD,1BD1BA1 Parking500-699 sqft,Sold in 2010,North York
2,"$304,500 $314,900","23 Sheppard Ave E, Unit 302",500-699 sqft,1+1,1+1BD1BA0 Parking500-699 sqft,Sold in 2010,North York
3,"$270,000 $278,999","18 Holmes Ave, Unit 2409",500-699 sqft,1BD,1BD1BA1 Parking500-699 sqft,Sold in 2010,North York
4,"$264,500 $265,800","22 Olive Ave, LPH10",,1BD,1BD1BA1 Parking,Sold in 2010,North York
...,...,...,...,...,...,...,...
6583,"$550,000 $524,900",1611 - 1360 York Mills Rd,500-599 sqft,3BD,"3BD2BA1 Parking1,000-1,199 sqft",Sold in 2021,North York
6584,"$585,000 $499,000",1114 - 8 Rean Dr,"1,000-1,199 sqft",1BD,1BD1BA1 Parking500-599 sqft,Sold in 2021,North York
6585,"$695,000 $699,999",106 - 1131 Steeles Ave,"1,400-1,599 sqft",2+1,"2+1BD2BA1 Parking1,000-1,199 sqft",Sold in 2021,North York
6586,"$825,000 $699,000",133 - 4001 Don Mills Rd,800-899 sqft,4BD,"4BD3BA1 Parking1,400-1,599 sqft",Sold in 2021,North York


In [21]:
yc_sold_prices=[]
yc_sold_location=[]
yc_sold_sqft=[]
yc_sold_bd=[]
yc_sold_all=[]
yc_sold_date=[]

def get_page(year,page):
    url= f'https://condos.ca/toronto/condos-for-sale?size_range=300%2C999999999&property_type=Condo%20Apt%2CComm%20Element%20Condo%2CLeasehold%20Condo&mode=Sold&end_date_unix=exact%2C{year}&sublocality_id=18&page={page}'
    driver.get(url) 
    sleep(3)
    page_source = driver.page_source
    soup = BeautifulSoup(page_source, 'lxml')
    return soup

for year in range(2010,2022):
    for page in range(12):
        soup = get_page(year,page)
        for tag in soup.find_all('div',class_ = 'styles___AskingPrice-sc-54qk44-4 styles___ClosePrice-sc-54qk44-5 dHPUdq hwkkXU'):
            yc_sold_prices.append(tag.get_text())

        for tag in soup.find_all('address',class_ = 'styles___Address-sc-54qk44-13 gTwVlm'):
            yc_sold_location.append(tag.get_text())


        for tag in soup.find_all(class_ = 'styles___Size-sc-54qk44-8 KRKbD'):
            yc_sold_sqft.append(tag.get_text())


        for tag in soup.find_all(class_ = 'styles___InfoHolder-sc-54qk44-7 jtFhfz'):
            bedrooms = tag.get_text()[0:3]
            yc_sold_bd.append(bedrooms)

        for tag in soup.find_all(class_ = 'styles___InfoHolder-sc-54qk44-7 jtFhfz'):
            everything = tag.get_text()[0:]
            yc_sold_all.append(everything)

        for tag in soup.find_all(class_ = 'styles___Banner-sc-1a2baz3-0 iWaXsf'):
            everything = tag.get_text()[0:]
            yc_sold_date.append(everything)

In [22]:
e = {'Prices':yc_sold_prices,
                         'Location':yc_sold_location,
                         'Square Ft':yc_sold_sqft,
                         'Bedrooms':yc_sold_bd,
                         'Everything':yc_sold_all,
                         'Sold Date':yc_sold_date}

yc_sold_condos = pd.DataFrame.from_dict(e, orient='index')
yc_sold_condos = yc_sold_condos.transpose()

yc_sold_condos['Region']= 'York Crosstown'
yc_sold_condos

,Prices,Location,Square Ft,Bedrooms,Everything,Sold Date,Region
0,"$162,500 $169,900","2464 Weston Rd, Unit 803",700-899 sqft,1BD,1BD1BA1 Parking700-899 sqft,Sold in 2010,York Crosstown
1,"$200,000 $204,000","360 Ridelle Ave, Unit 310",,2BD,2BD2BA1 Parking,Sold in 2010,York Crosstown
2,"$196,000 $198,900","3 Hickory Tree Rd, Unit 1001","1,100-1,299 sqft",2+1,"2+1BD2BA1 Parking1,100-1,299 sqft",Sold in 2010,York Crosstown
3,"$309,900 $309,900","1001 Roselawn Ave, Unit 210",700-899 sqft,1BD,1BD1BA1 Parking700-899 sqft,Sold in 2010,York Crosstown
4,"$149,000 $151,000","3 Hickory Tree Rd, Unit 2007",700-899 sqft,1+1,1+1BD1BA1 Parking700-899 sqft,Sold in 2010,York Crosstown
...,...,...,...,...,...,...,...
3365,"$635,000 $649,900",1703 - 75 Emmett Ave,"1,600-1,799 sqft",3+1,"3+1BD2BA1 Parking1,600-1,799 sqft",Sold in 2021,York Crosstown
3366,"$555,000 $568,800",508 - 75 Emmett Ave,"1,200-1,399 sqft",2+1,"2+1BD2BA1 Parking1,200-1,399 sqft",Sold in 2021,York Crosstown
3367,"$617,000 $640,000",201 - 1110 Briar Hill Ave,700-799 sqft,2BD,2BD1BA1 Parking700-799 sqft,Sold in 2021,York Crosstown
3368,"$590,000 $599,000",303 - 360 Ridelle Ave,"1,000-1,199 sqft",2BD,"2BD1BA1 Parking1,000-1,199 sqft",Sold in 2021,York Crosstown


In [23]:
we_sold_prices=[]
we_sold_location=[]
we_sold_sqft=[]
we_sold_bd=[]
we_sold_all=[]
we_sold_date=[]

def get_page(year,page):
    url= f'https://condos.ca/toronto/condos-for-sale?size_range=300%2C999999999&property_type=Condo%20Apt%2CComm%20Element%20Condo%2CLeasehold%20Condo&mode=Sold&end_date_unix=exact%2C{year}&sublocality_id=19&page={page}'
    driver.get(url) 
    sleep(3)
    page_source = driver.page_source
    soup = BeautifulSoup(page_source, 'lxml')
    return soup

for year in range(2010,2022):
    for page in range(12):
        soup = get_page(year,page)
        for tag in soup.find_all('div',class_ = 'styles___AskingPrice-sc-54qk44-4 styles___ClosePrice-sc-54qk44-5 dHPUdq hwkkXU'):
            we_sold_prices.append(tag.get_text())

        for tag in soup.find_all('address',class_ = 'styles___Address-sc-54qk44-13 gTwVlm'):
            we_sold_location.append(tag.get_text())


        for tag in soup.find_all(class_ = 'styles___Size-sc-54qk44-8 KRKbD'):
            we_sold_sqft.append(tag.get_text())


        for tag in soup.find_all(class_ = 'styles___InfoHolder-sc-54qk44-7 jtFhfz'):
            bedrooms = tag.get_text()[0:3]
            we_sold_bd.append(bedrooms)

        for tag in soup.find_all(class_ = 'styles___InfoHolder-sc-54qk44-7 jtFhfz'):
            everything = tag.get_text()[0:]
            we_sold_all.append(everything)

        for tag in soup.find_all(class_ = 'styles___Banner-sc-1a2baz3-0 iWaXsf'):
            everything = tag.get_text()[0:]
            we_sold_date.append(everything)

In [24]:
f = {'Prices':we_sold_prices,
                         'Location':we_sold_location,
                         'Square Ft':we_sold_sqft,
                         'Bedrooms':we_sold_bd,
                         'Everything':we_sold_all,
                         'Sold Date':we_sold_date}

we_sold_condos = pd.DataFrame.from_dict(f, orient='index')
we_sold_condos = we_sold_condos.transpose()

we_sold_condos['Region']= 'West End'
we_sold_condos

,Prices,Location,Square Ft,Bedrooms,Everything,Sold Date,Region
0,"$250,000 $244,900","100 Western Battery Rd, Unit 813",,1BD,1BD1BA0 Parking,Sold in 2010,West End
1,"$388,000 $409,000","43 Hanna Ave, Unit 313",700-899 sqft,1BD,1BD1BA1 Parking700-899 sqft,Sold in 2010,West End
2,"$244,900 $244,900","100 Western Battery Rd, Unit 713",,1BD,1BD1BA0 Parking,Sold in 2010,West End
3,"$320,000 $324,900","100 Western Battery Rd, Unit 202",500-699 sqft,1+1,1+1BD1BA1 Parking500-699 sqft,Sold in 2010,West End
4,"$430,000 $439,900","100 Western Battery Rd, Unit 1805",700-899 sqft,2BD,2BD2BA1 Parking700-899 sqft,Sold in 2010,West End
...,...,...,...,...,...,...,...
6583,"$640,000 $599,000",314 - 1420 Dupont St,500-599 sqft,2BD,2BD1BA1 Parking600-699 sqft,Sold in 2021,West End
6584,"$685,000 $599,000",401 - 369 Sorauren Ave,500-599 sqft,1BD,1BD1BA0 Parking500-599 sqft,Sold in 2021,West End
6585,"$1,335,000 $1,349,800",Gs3 - 243 Perth Ave,None,3+1,"3+1BD2BA1 Parking1,400-1,599 sqft",Sold in 2021,West End
6586,"$535,000 $539,990",1703E - 36 Lisgar St,None,2BD,2BD2BA1 Parking500-599 sqft,Sold in 2021,West End


In [25]:
ee_sold_prices=[]
ee_sold_location=[]
ee_sold_sqft=[]
ee_sold_bd=[]
ee_sold_all=[]
ee_sold_date=[]

def get_page(year,page):
    url= f'https://condos.ca/toronto/condos-for-sale?size_range=300%2C999999999&property_type=Condo%20Apt%2CComm%20Element%20Condo%2CLeasehold%20Condo&mode=Sold&end_date_unix=exact%2C{year}&sublocality_id=20&page={page}'
    driver.get(url) 
    sleep(3)
    page_source = driver.page_source
    soup = BeautifulSoup(page_source, 'lxml')
    return soup

for year in range(2010,2022):
    for page in range(12):
        soup = get_page(year,page)
        for tag in soup.find_all('div',class_ = 'styles___AskingPrice-sc-54qk44-4 styles___ClosePrice-sc-54qk44-5 dHPUdq hwkkXU'):
            ee_sold_prices.append(tag.get_text())

        for tag in soup.find_all('address',class_ = 'styles___Address-sc-54qk44-13 gTwVlm'):
            ee_sold_location.append(tag.get_text())


        for tag in soup.find_all(class_ = 'styles___Size-sc-54qk44-8 KRKbD'):
            ee_sold_sqft.append(tag.get_text())


        for tag in soup.find_all(class_ = 'styles___InfoHolder-sc-54qk44-7 jtFhfz'):
            bedrooms = tag.get_text()[0:3]
            ee_sold_bd.append(bedrooms)

        for tag in soup.find_all(class_ = 'styles___InfoHolder-sc-54qk44-7 jtFhfz'):
            everything = tag.get_text()[0:]
            ee_sold_all.append(everything)

        for tag in soup.find_all(class_ = 'styles___Banner-sc-1a2baz3-0 iWaXsf'):
            everything = tag.get_text()[0:]
            ee_sold_date.append(everything)

In [26]:
g = {'Prices':ee_sold_prices,
                         'Location':ee_sold_location,
                         'Square Ft':ee_sold_sqft,
                         'Bedrooms':ee_sold_bd,
                         'Everything':ee_sold_all,
                         'Sold Date':ee_sold_date}

ee_sold_condos = pd.DataFrame.from_dict(g, orient='index')
ee_sold_condos = ee_sold_condos.transpose()

ee_sold_condos['Region']= 'East End'
ee_sold_condos

,Prices,Location,Square Ft,Bedrooms,Everything,Sold Date,Region
0,"$297,000 $299,500","2112 Queen St E, Unit 27",,1BD,1BD1BA1 Parking,Sold in 2010,East End
1,"$395,000 $398,000","1765 Queen St E, Unit 405",,2BD,2BD2BA1 Parking,Sold in 2010,East End
2,"$465,000 $475,000","68 Broadview Ave, Unit 116","1,100-1,299 sqft",2BD,"2BD1BA1 Parking1,100-1,299 sqft",Sold in 2010,East End
3,"$328,000 $335,000","201 Carlaw Ave, Unit 510",,1BD,1BD1BA1 Parking,Sold in 2010,East End
4,"$441,000 $435,000","233 Carlaw Ave, Unit 406","900-1,099 sqft",2BD,"2BD1BA1 Parking900-1,099 sqft",Sold in 2010,East End
...,...,...,...,...,...,...,...
3631,"$575,000 $589,900",525 - 2301 Danforth Ave,600-699 sqft,1+1,1+1BD1BA1 Parking600-699 sqft,Sold in 2021,East End
3632,"$665,000 $689,900",306 - 601 Kingston Rd,700-799 sqft,1+1,1+1BD1BA1 Parking700-799 sqft,Sold in 2021,East End
3633,"$500,000 $529,000",614 - 2301 Danforth Ave,500-599 sqft,1+1,1+1BD1BA1 Parking500-599 sqft,Sold in 2021,East End
3634,"$891,000 $879,000",213 - 60 Haslett Ave,900-999 sqft,2+1,2+1BD2BA1 Parking900-999 sqft,Sold in 2021,East End


In [27]:
mt_sold_prices=[]
mt_sold_location=[]
mt_sold_sqft=[]
mt_sold_bd=[]
mt_sold_all=[]
mt_sold_date=[]

def get_page(year,page):
    url= f'https://condos.ca/toronto/condos-for-sale?size_range=300%2C999999999&property_type=Condo%20Apt%2CComm%20Element%20Condo%2CLeasehold%20Condo&mode=Sold&end_date_unix=exact%2C{year}&sublocality_id=21&page={page}'
    driver.get(url) 
    sleep(3)
    page_source = driver.page_source
    soup = BeautifulSoup(page_source, 'lxml')
    return soup

for year in range(2010,2022):
    for page in range(12):
        soup = get_page(year,page)
        for tag in soup.find_all('div',class_ = 'styles___AskingPrice-sc-54qk44-4 styles___ClosePrice-sc-54qk44-5 dHPUdq hwkkXU'):
            mt_sold_prices.append(tag.get_text())

        for tag in soup.find_all('address',class_ = 'styles___Address-sc-54qk44-13 gTwVlm'):
            mt_sold_location.append(tag.get_text())


        for tag in soup.find_all(class_ = 'styles___Size-sc-54qk44-8 KRKbD'):
            mt_sold_sqft.append(tag.get_text())


        for tag in soup.find_all(class_ = 'styles___InfoHolder-sc-54qk44-7 jtFhfz'):
            bedrooms = tag.get_text()[0:3]
            mt_sold_bd.append(bedrooms)

        for tag in soup.find_all(class_ = 'styles___InfoHolder-sc-54qk44-7 jtFhfz'):
            everything = tag.get_text()[0:]
            mt_sold_all.append(everything)

        for tag in soup.find_all(class_ = 'styles___Banner-sc-1a2baz3-0 iWaXsf'):
            everything = tag.get_text()[0:]
            mt_sold_date.append(everything)

In [28]:
h = {'Prices':mt_sold_prices,
                         'Location':mt_sold_location,
                         'Square Ft':mt_sold_sqft,
                         'Bedrooms':mt_sold_bd,
                         'Everything':mt_sold_all,
                         'Sold Date':mt_sold_date}

mt_sold_condos = pd.DataFrame.from_dict(h, orient='index')
mt_sold_condos = mt_sold_condos.transpose()

mt_sold_condos['Region']= 'Midtown'
mt_sold_condos

,Prices,Location,Square Ft,Bedrooms,Everything,Sold Date,Region
0,"$412,000 $429,900","188 Eglinton Ave E, Unit 208","1,100-1,299 sqft",2BD,"2BD2BA1 Parking1,100-1,299 sqft",Sold in 2010,Midtown
1,"$241,000 $259,000","380 Macpherson Ave, Unit 228",500-699 sqft,Stu,Studio1BA0 Parking500-699 sqft,Sold in 2010,Midtown
2,"$660,000 $679,000","25 Broadway Ave, Unit 1602","1,100-1,299 sqft",2+1,"2+1BD2BA1 Parking1,100-1,299 sqft",Sold in 2010,Midtown
3,"$435,000 $439,900","25 Broadway Ave, Unit 1805",700-899 sqft,1+1,1+1BD1BA1 Parking700-899 sqft,Sold in 2010,Midtown
4,"$383,000 $389,000","650 Mount Pleasant Rd, Unit 306",700-899 sqft,1+1,1+1BD2BA1 Parking700-899 sqft,Sold in 2010,Midtown
...,...,...,...,...,...,...,...
6583,"$575,000 $589,000",218 - 223 St Clair Ave W,500-599 sqft,1BD,1BD1BA1 Parking500-599 sqft,Sold in 2021,Midtown
6584,"$743,000 $679,000",312 - 377 Madison Ave,700-799 sqft,2BD,2BD2BA1 Parking700-799 sqft,Sold in 2021,Midtown
6585,"$613,000 $639,900",4704 - 8 Eglinton Ave E,500-599 sqft,1+1,1+1BD1BA0 Parking500-599 sqft,Sold in 2021,Midtown
6586,"$870,000 $849,000",610 - 35 Merton St,"1,000-1,199 sqft",2BD,"2BD2BA1 Parking1,000-1,199 sqft",Sold in 2021,Midtown


In [30]:
ey_sold_prices=[]
ey_sold_location=[]
ey_sold_sqft=[]
ey_sold_bd=[]
ey_sold_all=[]
ey_sold_date=[]

def get_page(year,page):
    url= f'https://condos.ca/toronto/condos-for-sale?size_range=300%2C999999999&property_type=Condo%20Apt%2CComm%20Element%20Condo%2CLeasehold%20Condo&mode=Sold&end_date_unix=exact%2C{year}&sublocality_id=22&page={page}'
    driver.get(url) 
    sleep(3)
    page_source = driver.page_source
    soup = BeautifulSoup(page_source, 'lxml')
    return soup

for year in range(2010,2022):
    for page in range(12):
        soup = get_page(year,page)
        for tag in soup.find_all('div',class_ = 'styles___AskingPrice-sc-54qk44-4 styles___ClosePrice-sc-54qk44-5 dHPUdq hwkkXU'):
            ey_sold_prices.append(tag.get_text())

        for tag in soup.find_all('address',class_ = 'styles___Address-sc-54qk44-13 gTwVlm'):
            ey_sold_location.append(tag.get_text())


        for tag in soup.find_all(class_ = 'styles___Size-sc-54qk44-8 KRKbD'):
            ey_sold_sqft.append(tag.get_text())


        for tag in soup.find_all(class_ = 'styles___InfoHolder-sc-54qk44-7 jtFhfz'):
            bedrooms = tag.get_text()[0:3]
            ey_sold_bd.append(bedrooms)

        for tag in soup.find_all(class_ = 'styles___InfoHolder-sc-54qk44-7 jtFhfz'):
            everything = tag.get_text()[0:]
            ey_sold_all.append(everything)

        for tag in soup.find_all(class_ = 'styles___Banner-sc-1a2baz3-0 iWaXsf'):
            everything = tag.get_text()[0:]
            ey_sold_date.append(everything)

In [31]:
j = {'Prices':ey_sold_prices,
                         'Location':ey_sold_location,
                         'Square Ft':ey_sold_sqft,
                         'Bedrooms':ey_sold_bd,
                         'Everything':ey_sold_all,
                         'Sold Date':ey_sold_date}

ey_sold_condos = pd.DataFrame.from_dict(j, orient='index')
ey_sold_condos = ey_sold_condos.transpose()

ey_sold_condos['Region']= 'East York'
ey_sold_condos

,Prices,Location,Square Ft,Bedrooms,Everything,Sold Date,Region
0,"$172,000 $178,000","4 Park Vista Dr, Unit 604",700-899 sqft,1+1,1+1BD1BA1 Parking700-899 sqft,Sold in 2010,East York
1,"$212,000 $214,999","21 Overlea Blvd, PH 01",,1+1,1+1BD1BA1 Parking,Sold in 2010,East York
2,"$259,000 $259,000","1048 Broadview Ave, Unit 2006",500-699 sqft,1BD,1BD1BA1 Parking500-699 sqft,Sold in 2010,East York
3,"$345,000 $349,900","356 Mcrae Dr, Unit 108",700-899 sqft,1+1,1+1BD1BA1 Parking700-899 sqft,Sold in 2010,East York
4,"$179,000 $182,900","11 Thorncliffe Park Dr, PH 17",,1BD,1BD1BA1 Parking,Sold in 2010,East York
...,...,...,...,...,...,...,...
2856,"$665,000 $679,000",406 - 980 Broadview Ave,"1,000-1,199 sqft",2BD,"2BD2BA1 Parking1,000-1,199 sqft",Sold in 2021,East York
2857,"$840,000 $859,990",1206 - 35 Brian Peck Cres,"1,000-1,199 sqft",2+1,"2+1BD2BA2 Parking1,000-1,199 sqft",Sold Conditional in 2021,East York
2858,"$440,000 $449,000",1807 - 3 Massey Sq,"1,000-1,199 sqft",2BD,"2BD1BA0 Parking1,000-1,199 sqft",Sold in 2021,East York
2859,"$425,000 $439,900",1111 - 5 Massey Sq,900-999 sqft,2BD,2BD1BA1 Parking900-999 sqft,Sold Conditional in 2021,East York


In [32]:
frames2 = [df_sold_condos, sc_sold_condos,
et_sold_condos,ny_sold_condos,yc_sold_condos,we_sold_condos,ee_sold_condos, 
mt_sold_condos, ey_sold_condos]

df_condos_sold = pd.concat(frames2,axis=0,sort=True)
print(df_condos_sold.head())

  Bedrooms                         Everything                     Location  \
0      2+1  2+1BD2BA1 Parking1,500-2,000 sqft      80 Front St E, Unit 718   
1      2BD      2BD2BA1 Parking900-1,099 sqft       628 Fleet St, Unit 414   
2      2BD        2BD2BA1 Parking700-899 sqft     21 Carlton St, Unit 1405   
3      1BD        1BD1BA0 Parking500-699 sqft  110 Charles St E, Unit 2305   
4      2BD      2BD2BA1 Parking900-1,099 sqft     18 Beverley St, Unit 109   

               Prices    Region     Sold Date         Square Ft  
0  $845,000  $899,000  Downtown  Sold in 2010  1,500-2,000 sqft  
1  $458,000  $478,000  Downtown  Sold in 2010    900-1,099 sqft  
2  $450,000  $460,000  Downtown  Sold in 2010      700-899 sqft  
3  $355,000  $369,900  Downtown  Sold in 2010      500-699 sqft  
4  $347,500  $355,000  Downtown  Sold in 2010    900-1,099 sqft  


In [33]:
df_condos_sold.to_excel('df_condos_sold.xlsx')

In [34]:
#To clean up the Everything Column in order to pull out the number of Parking
df_condos_sold[['A','B']] = df_condos_sold['Everything'].str.split(' ',1, expand=True)
df_condos_sold['Parking'] = df_condos_sold['A'].str.strip().str[-1]
df_condos_sold = df_condos_sold.drop(columns = ['Everything','A','B'])

In [35]:
df_condos_sold['Prices'] = df_condos_sold['Prices'].str.replace('$','')
df_condos_sold['Prices'] = df_condos_sold['Prices'].str.replace(',','')
df_condos_sold[['Listed Price','Sold Price']] = df_condos_sold['Prices'].str.split(' ',1,expand=True)

In [36]:
#To clean up the sq ft column
df_condos_sold['Square Ft'] = df_condos_sold['Square Ft'].str.replace('sqft','')

#Remove the dash and split the min and max into two columns.
df_condos_sold[['Min Sq Ft','Max Sq Ft']] = df_condos_sold['Square Ft'].str.split('-',1, expand=True)

#Clean up the Min Sq Ft column
df_condos_sold['Min Sq Ft'] = df_condos_sold['Min Sq Ft'].str.replace(',','')

#Clean up the commas in the Max column
df_condos_sold['Max Sq Ft'] = df_condos_sold['Max Sq Ft'].str.replace(',','')

In [37]:
#Replace any blanks with nan in the Min Sq Ft Column
df_condos_sold['Min Sq Ft'].replace('',np.nan,inplace=True)

In [38]:
#Replace any blanks with nan in the Max Sq Ft Column
df_condos_sold['Max Sq Ft'].replace('',np.nan,inplace=True)

In [39]:
#Replace any Min and Max Sq Ft.values that are 0, with Min or Max Sq Ft Values.
df_condos_sold['Min Sq Ft'] = df_condos_sold['Min Sq Ft'].mask(df_condos_sold['Min Sq Ft']==0).fillna(df_condos_sold['Max Sq Ft'])
df_condos_sold['Max Sq Ft'] = df_condos_sold['Max Sq Ft'].mask(df_condos_sold['Max Sq Ft']==0).fillna(df_condos_sold['Min Sq Ft'])

In [40]:
#Drop any remaining nulls
df_condos_sold.dropna(subset = ['Max Sq Ft'],inplace=True)

In [41]:
#Initialize a new column to calculate Average Square Feet. 
#This column will be used to average the min/max square feet.
df_condos_sold['Avg Sq Ft'] = 0

In [42]:
#Convert columns to integers so they can be calculated later
df_condos_sold['Sold Price'] = df_condos_sold['Sold Price'].astype(int)

In [43]:
#Convert columns to integers so they can be calculated later
df_condos_sold['Min Sq Ft'] = df_condos_sold['Min Sq Ft'].astype(int)

In [44]:
#Convert columns to integers so they can be calculated later
df_condos_sold['Max Sq Ft'] = df_condos_sold['Max Sq Ft'].astype(int)

In [45]:
#Replace any Min and Max Sq Ft.values that are 0, with Min or Max Sq Ft Values.
df_condos_sold['Min Sq Ft'] = df_condos_sold['Min Sq Ft'].mask(df_condos_sold['Min Sq Ft']==0).fillna(df_condos_sold['Max Sq Ft'])
df_condos_sold['Max Sq Ft'] = df_condos_sold['Max Sq Ft'].mask(df_condos_sold['Max Sq Ft']==0).fillna(df_condos_sold['Min Sq Ft'])

In [46]:
#Fix the Max Sq Ft Error. There is a typo in one of the dataset fields
df_condos_sold['Max Sq Ft'] = df_condos_sold['Max Sq Ft'].replace({65535:5000})

In [47]:
#Take the avg Sq ft per unit
df_condos_sold['Avg Sq Ft'] = (df_condos_sold['Min Sq Ft'] + df_condos_sold['Max Sq Ft'])/2

In [48]:
#Calculate the Avg Price per Square Ft
df_condos_sold['Avg Price per Sq Ft'] = 0
df_condos_sold['Avg Price per Sq Ft'] = df_condos_sold['Sold Price']/df_condos_sold['Avg Sq Ft']

In [49]:
#From web scraping, there are some text fields in the Sold Dates. Remove these text values.
df_condos_sold['Sold Date'] = df_condos_sold['Sold Date'].str.replace('Sold in','')

In [50]:
#From web scraping, there are some text fields in the Sold Dates. Remove these text values.
df_condos_sold['Sold Date'] = df_condos_sold['Sold Date'].str.replace('Sold Conditional in','')

In [53]:
df_condos_sold.to_excel('HistoricalCondoPrices_Toronto.xlsx')